In [1]:
%matplotlib inline
import pandas as pd
import torch
import torch.nn as nn
from matplotlib import pyplot as plt

In [2]:
torch.set_default_tensor_type('torch.FloatTensor')
print(torch.__version__)

1.10.0+cu113


In [3]:
# Load the data
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

In [4]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
print(train_data.shape, test_data.shape)

(1460, 81) (1459, 80)


In [6]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))
train_features = all_features.iloc[:train_data.shape[0], 1:-1]
train_labels = all_features.iloc[:train_data.shape[0], -1]

### 数据处理

In [7]:
# 数值数据标准化
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
# 填充均值0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [8]:
all_features = pd.get_dummies(all_features, dummy_na=True)
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32).to('cuda')
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32).to('cuda')
train_labels = torch.tensor(train_data.SalePrice.values.reshape(-1, 1), dtype=torch.float32).to('cuda')

In [9]:
# loss
loss = nn.MSELoss()


def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)


def get_net(feature_num):
    net = nn.Sequential(
        nn.Flatten(), nn.Linear(feature_num, 1)
    )
    net.apply(init_weights)
    return net

In [10]:
# 对数误差
def log_rmse(net, features, label):
    clipped_preds = torch.clamp(net(features), min=1.0, max=float('inf'))
    rmse = torch.sqrt(loss(clipped_preds.log(), label.log()))
    return rmse.item()

In [11]:
def train(net, train_features, train_label, test_features, test_label, batch_size, epoch, lr, wd):
    dataset = torch.utils.data.TensorDataset(train_features, train_label)
    train_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)
    net = net.float()  #32位
    net = net.to('cuda')
    optimizer = torch.optim.Adam(params=net.parameters(), lr=lr, weight_decay=wd)
    train_ls, test_ls = [], []
    for _ in range(epoch):
        for X, Y in train_iter:
            optimizer.zero_grad()
            train_pred = net(X.float())
            train_loss = loss(train_pred, Y.float())
            train_loss.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_label))
        if test_label is not None:
            test_ls.append(log_rmse(net, test_features, test_label))
    return train_ls, test_ls

### K折交叉验证

In [12]:
def get_k_fold_data(k, i, X, Y):
    # 第i折作为测试集，其余作为训练集
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, Y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, Y_part = X[idx, :], Y[idx]
        if j == i:
            X_valid, Y_valid = X_part, Y_part
        elif X_train is None:
            X_train, Y_train = X_part, Y_part
        else:
            X_train = torch.cat((X_train, X_part), dim=0)
            Y_train = torch.cat((Y_train, Y_part), dim=0)
    return X_train, Y_train, X_valid, Y_valid

In [13]:
def k_fold(k, X_train, Y_train, batch_size, epoch, lr, wd):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        X_train, Y_train, X_valid, Y_valid = get_k_fold_data(k, i, X_train, Y_train)
        net = get_net(X_train.shape[1])
        train_ls, valid_ls = train(net, X_train, Y_train, X_valid, Y_valid, batch_size, epoch, lr, wd)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        print(f'{i + 1}折，训练误差：{train_ls[-1]}, 验证误差：{valid_ls[-1]}')
    return train_l_sum / k, valid_l_sum / k

In [14]:
k, epoch, lr, wd, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, batch_size, epoch, lr, wd)

1折，训练误差：0.16949453949928284, 验证误差：0.1564026176929474
2折，训练误差：0.17200008034706116, 验证误差：0.1909938007593155
3折，训练误差：0.1986875683069229, 验证误差：0.19064760208129883
4折，训练误差：0.2422313541173935, 验证误差：0.27702102065086365
5折，训练误差：0.34421470761299133, 验证误差：0.34014931321144104


In [15]:
def train_and_pred(train_features, train_labels, test_features, batch_size, epochs, lr, wd):
    net = get_net(train_features.shape[1]).to('cuda')
    train_ls, _ = train(net, train_features, train_labels, None, None, batch_size, epochs, lr, wd)
    print(f'训练误差：{train_ls[-1]}')
    plt.plot(train_ls,label='epoch')
    plt.yscale("log")
    plt.legend();
    test_preds = net(test_features).detach().numpy()
    test_features['SalePrice'] = pd.Series(test_data.reshape(1, -1)[0])
    submission = pd.concat([test_features['Id'], test_features['SalePrice']], axis=1)
    submission.to_csv('./data/submission.csv', index=False)
    print('finish')
    return test_preds

In [ ]:
train_and_pred(train_features, train_labels, test_features, batch_size, epoch, lr, wd)

训练误差：0.16240538656711578
